In [4]:
import json
import pandas as pd

In [2]:
# !pip install .

In [3]:
!pip install simplet5 

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)


  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.0
    Uninstalling transformers-4.19.0:
      Successfully uninstalled transformers-4.19.0
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [4]:
!myquota


Hostname: gr015 at Thu May 12 19:11:50 EDT 2022

Filesystem   Environment   Backed up?   Allocation       Current Usage
Space        Variable      /Flushed?    Space / Files    Space(%) / Files(%)

/home        $HOME         Yes/No       50.0GB/30.0K      10.96GB(21.92%)/21483(71.61%)
/scratch     $SCRATCH      No/Yes        5.0TB/1.0M     141.04GB(2.75%)/97135(9.71%)



In [5]:
!pip install transformers --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.19.0-py3-none-any.whl (4.2 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

simplet5 0.1.4 requires transformers==4.16.2, but you'll have transformers 4.19.0 which is incompatible.
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [6]:
# !pip install .

In [7]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-22.1-py3-none-any.whl (2.1 MB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [8]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import random
import numpy as np
import torch 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [7]:
import numpy as np
import pandas as pd
# train=pd.read_json('/content/drive/MyDrive/NYU courses/Sem 2/NLU/Datasets/News_dataset/Sarcasm_Headlines_Dataset_v2.json',lines=True)
train=pd.read_json('/scratch/ka2522/NLU_S22_Project/News_dataset/Sarcasm_Headlines_Dataset_v2.json',lines=True)
train = train.rename(columns={'is_sarcastic': 'target_text', 'headline': 'source_text'})
for index, row in train.iterrows():
    if train.iloc[index]['target_text']==1:
        train.replace(to_replace=1, value =  'sarcasm', inplace=True)
    else:
        train.replace(to_replace=0,value =  'not sarcasm',inplace=True)



In [12]:
# from tqdm import tqdm
# m = -1
# mi=1000000000000
# for index, row in tqdm(train.iterrows()):
#     m = max(m,len(row['source_text']))
#     mi=min(mi,len(row['source_text']))
    
# print(m,mi)

In [13]:
train=train.drop(['article_link'],axis=1)
train.head()


,target_text,source_text
0,sarcasm,thirtysomething scientists unveil doomsday clo...
1,not sarcasm,dem rep. totally nails why congress is falling...
2,not sarcasm,eat your veggies: 9 deliciously different recipes
3,sarcasm,inclement weather prevents liar from getting t...
4,sarcasm,mother comes pretty close to using word 'strea...


In [8]:
import random

train = train.sample(frac = 1)
train_data = train[:17171]
test_data = train[17171:22895]
val_data = train[22895:]

In [15]:
from simplet5 import SimpleT5

Global seed set to 42


In [16]:
model = SimpleT5()
model.from_pretrained("byt5","google/byt5-base")

Downloading:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/721 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [17]:
# train_data.size

In [9]:
model.train(train_df=train_data, 
            eval_df=val_data, 
            source_max_token_len = 600, 
            target_max_token_len = 128,
            batch_size = 2,
            max_epochs = 3,
            use_gpu = True,
            outputdir = "Byt5_base_epoch_3_outputs",
            early_stopping_patience_epochs = 0
            )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 581 M 
-----------------------------------------------------
581 M     Trainable params
0         Non-trainable params
581 M     Total params
2,326.613 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/ka2522/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/ka2522/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# Epoch 2 11448/11448 [1:26:19<00:00, 2.21it/s, loss=0.05, v_num=1.95e+7, train_loss_step=0.0104, val_loss_step=0.021, val_loss_epoch=0.0345, train_loss_epoch=0.0362]

In [1]:
from simplet5 import SimpleT5
model = SimpleT5()

Global seed set to 42


In [2]:
model.load_model('byt5', '/scratch/ka2522/NLU_S22_Project/Byt5_base_epoch_3_outputs/simplet5-epoch-2-train-loss-0.0362-val-loss-0.0345')

In [23]:
test_data.head()
x = model.predict("fwf")
print(x)

['not sarcasm']


In [24]:
lis=[]
from tqdm import tqdm
count=0
value=0
for index, row in tqdm(test_data.iterrows()):
    ans = model.predict(row['source_text'])
    if ans[0]==row['target_text']:
        count+=1
    if value%1000==0:
        print(value,count)
    value+=1
    lis.append(ans[0])

print(count*100/5724)

1it [00:00,  1.32it/s]

0 1


1001it [12:08,  1.42it/s]

1000 853


2001it [24:17,  1.39it/s]

...

4000 3423


5001it [1:00:29,  1.42it/s]

5000 4285


5724it [1:09:16,  1.38it/s]

85.58700209643605


In [25]:
print("Accuracy on 3 epochs",count*100/5724)

Accuracy on 3 epochs 85.58700209643605
